In [7]:
import sys
import pickle
import numpy as np
import pandas as pd

sys.path.append('../src')
sys.path.append('../src/data')

from design_gene_blocks import DesignEblocks

In [8]:
mutations = 'files/mutations_ed.txt'
vector = 'files/snapgene_vector.dna'
gene = 'files/mtb_DnaE1_seq.txt'
output_location = 'output/'
species = 'mtb' # Mycobacterium Tuberculosis
codon_usage = 'src/data/codon_usage/'

In [9]:
# Create instance of DesignEblocks

eBlocks_instance = DesignEblocks(
                        sequence_fp=gene,
                        mutations_fp=mutations,
                        output_fp=output_location,
                        species='mtb')



G432E Mutation
R436Q Mutation
I451A Mutation
I477D Mutation
I477D-A484S Combined
A484S Mutation
I537-K562 Deletion
A540L Mutation
K562A Mutation
I569V Mutation
L584V Mutation
T595S Mutation
D599Y Mutation
P618D Mutation
N677H Mutation
Q722V Mutation
A770-PLR Insert
D773E Mutation
K922R Mutation
E972G Mutation
W973S Mutation
Q1023R Mutation
A1031T Mutation
N1034Q Mutation
N1038T Mutation
S1046N Mutation
L1052E Mutation
V1086N Mutation
R1090A Mutation
N1096E Mutation


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/codon_usage'